In [1]:
import pandas as pd

In [2]:
df_ans = pd.read_csv("../preprocess_test_data.csv",index_col=0)
df_ans['text_clean'] = df_ans['text_clean'].fillna('neutral')
df_ans.head()

,_id,text,task_1,task_2,task_3,task_4,text_clean
0,Gujarati_image_1225.jpg,॥વિંદેશીગામડિયો\r\n અ |,Non-Sarcastic,Neutral,Non Vulgar,Non-abusive,॥વિંદેશીગામડિયો\r\n અ |
1,Gujarati_image_1583.jpg,ટીચર : સૌથી વધારે દુખાવો ક્યારે\r\nથાય?\r\nછોક...,Sarcastic,Negative,Vulgar,Abusive,ટીચર : સૌથી વધારે દુખાવો ક્યારે\r\nથાય?\r\nછોક...
2,Gujarati_image_1502.jpg,પતિ: તુંમને જરાય પ્રેમ\r\nનથી કરતી...\r\n\r\nપ...,Sarcastic,Positive,Vulgar,Abusive,પતિ: તુંમને જરાય પ્રેમ\r\nનથી કરતી...\r\n\r\nપ...
3,Gujarati_image_1487.jpg,આખા ગોમ ના લોડા\r\nભોસ મા ભરી ને બેઠી\r\nહોય અ...,Sarcastic,Negative,Vulgar,Abusive,આખા ગોમ ના લોડા\r\nભોસ મા ભરી ને બેઠી\r\nહોય અ...
4,Gujarati_image_1497.jpg,મિનરલ વોટર સિવાય ક્યારેય\r\nબીજું\r\nપાણી નો પ...,Non-Sarcastic,Negative,Vulgar,Abusive,મિનરલ વોટર સિવાય ક્યારેય\r\nબીજું\r\nપાણી નો પ...


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=2)
X_test = tfidfconverter.fit_transform(df_ans['text_clean']).toarray()

In [4]:
import pickle
# 加载模型和特征转换器
with open('vectorizer.pkl', 'rb') as f:
    tfidfconverter = pickle.load(f)
X_test = tfidfconverter.transform(df_ans['text_clean']).toarray()

# task_1

In [5]:
import pickle
kn1= pickle.load(open('best_KN_model1.sav', 'rb'))
lr1= pickle.load(open('best_LR_model1.sav', 'rb'))
nb1= pickle.load(open('best_NB_model1.sav', 'rb'))
rf1= pickle.load(open('best_RF_model1.sav', 'rb'))
svm1= pickle.load(open('best_SVM_model1.sav', 'rb'))

In [6]:
X_test = X_test[:,:1741]

In [7]:
def transform(res):
    for i in range(len(res)):
        if (res[i] == 'Sarcastic'):
            res[i] = 0
        else:
            res[i] = 1
    return res

In [8]:
pred_df = pd.DataFrame() 
pred_kn = kn1.predict(X_test)
pred_kn = transform(pred_kn)
pred_df['KNeighbours'] = pred_kn
pred_rft = rf1.predict(X_test)
pred_rft = transform(pred_rft)
pred_df['RandomForest'] = pred_rft
pred_svm = svm1.predict(X_test)
pred_svm = transform(pred_svm)
pred_df['SVM'] = pred_svm

In [9]:
pred_kn.tolist().count(1),pred_rft.tolist().count(1),pred_svm.tolist().count(1)

(9, 39, 71)

In [10]:
df_ans = df_ans.drop(['text','text_clean'],axis=1)
df_ans = df_ans.reset_index()
df_ans = df_ans.drop(['index'],axis=1)
df_ans = df_ans.drop(['task_1','task_2','task_3','task_4'],axis=1)
df_ans.head()

,_id
0,Gujarati_image_1225.jpg
1,Gujarati_image_1583.jpg
2,Gujarati_image_1502.jpg
3,Gujarati_image_1487.jpg
4,Gujarati_image_1497.jpg


In [11]:
df_ans['label'] = pd.Series(pred_rft)
df_ans.to_csv('tfidf_rft.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,0
1,Gujarati_image_1583.jpg,1
2,Gujarati_image_1502.jpg,0
3,Gujarati_image_1487.jpg,0
4,Gujarati_image_1497.jpg,0
5,Gujarati_image_1623.jpg,0
6,Gujarati_image_1493.jpg,0
7,Gujarati_image_1492.jpg,0
8,Gujarati_image_1462.jpg,0
9,Gujarati_image_1500.jpg,0


In [12]:
df_ans['label'] = pd.Series(pred_svm)
df_ans.to_csv('tfidf_svm.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,0
1,Gujarati_image_1583.jpg,1
2,Gujarati_image_1502.jpg,0
3,Gujarati_image_1487.jpg,0
4,Gujarati_image_1497.jpg,0
5,Gujarati_image_1623.jpg,0
6,Gujarati_image_1493.jpg,1
7,Gujarati_image_1492.jpg,0
8,Gujarati_image_1462.jpg,0
9,Gujarati_image_1500.jpg,0


In [13]:
pred_df['res'] = pred_df.mode(axis=1)[0].astype(int)
pred_df.head()
res = pred_df['res'].tolist()

In [14]:
pred_df.head()

,KNeighbours,RandomForest,SVM,res
0,0,0,0,0
1,0,1,1,1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


In [15]:
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Sarcastic'
    else:
        res[i] = 'Non-Sarcastic'

In [16]:
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Non-Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Sarcastic


In [17]:
df_ans.to_csv('tfidf_overall.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Non-Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Sarcastic
5,Gujarati_image_1623.jpg,Sarcastic
6,Gujarati_image_1493.jpg,Sarcastic
7,Gujarati_image_1492.jpg,Sarcastic
8,Gujarati_image_1462.jpg,Sarcastic
9,Gujarati_image_1500.jpg,Sarcastic


In [18]:
res= pred_rft.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i]='Sarcastic'
    else:
        res[i]='Non-Sarcastic'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Non-Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Sarcastic


In [19]:
df_ans.to_csv('tfidf_rft.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Non-Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Sarcastic
5,Gujarati_image_1623.jpg,Sarcastic
6,Gujarati_image_1493.jpg,Sarcastic
7,Gujarati_image_1492.jpg,Sarcastic
8,Gujarati_image_1462.jpg,Sarcastic
9,Gujarati_image_1500.jpg,Sarcastic


In [20]:
res= pred_svm.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i]='Sarcastic'
    else:
        res[i]='Non-Sarcastic'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Non-Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Sarcastic


In [21]:
df_ans.to_csv('tfidf_svm.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Non-Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Sarcastic
5,Gujarati_image_1623.jpg,Sarcastic
6,Gujarati_image_1493.jpg,Non-Sarcastic
7,Gujarati_image_1492.jpg,Sarcastic
8,Gujarati_image_1462.jpg,Sarcastic
9,Gujarati_image_1500.jpg,Sarcastic


In [22]:
res = pred_kn.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Sarcastic'
    else:
        res[i] = 'Non-Sarcastic'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Sarcastic


In [23]:
df_ans.to_csv('tfidf_knc.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Sarcastic
1,Gujarati_image_1583.jpg,Sarcastic
2,Gujarati_image_1502.jpg,Sarcastic
3,Gujarati_image_1487.jpg,Sarcastic
4,Gujarati_image_1497.jpg,Sarcastic
5,Gujarati_image_1623.jpg,Sarcastic
6,Gujarati_image_1493.jpg,Sarcastic
7,Gujarati_image_1492.jpg,Sarcastic
8,Gujarati_image_1462.jpg,Sarcastic
9,Gujarati_image_1500.jpg,Sarcastic


# task_3

In [5]:
import pickle
kn3= pickle.load(open('best_KN_model3.sav', 'rb'))
lr3= pickle.load(open('best_LR_model3.sav', 'rb'))
nb3= pickle.load(open('best_NB_model3.sav', 'rb'))
rf3= pickle.load(open('best_RF_model3.sav', 'rb'))
svm3= pickle.load(open('best_SVM_model3.sav', 'rb'))

In [6]:
X_test = X_test[:,:1741]

In [7]:
def transform(res):
    for i in range(len(res)):
        if (res[i] == 'Vulgar'):
            res[i]= 0
        else:
            res[i]= 1
    return res

In [8]:
pred_df = pd.DataFrame() 
pred_kn = kn3.predict(X_test)
pred_kn = transform(pred_kn)
pred_df['KNeighbours'] = pred_kn
pred_rft = rf3.predict(X_test)
pred_rft = transform(pred_rft)
pred_df['RandomForest'] = pred_rft
pred_svm = svm3.predict(X_test)
pred_svm = transform(pred_svm)
pred_df['SVM'] = pred_svm

In [9]:
pred_kn.tolist().count(1),pred_rft.tolist().count(1),pred_svm.tolist().count(1)

(558, 452, 449)

In [10]:
df_ans = df_ans.drop(['text','text_clean'],axis=1)
df_ans = df_ans.reset_index()
df_ans = df_ans.drop(['index'],axis=1)
df_ans = df_ans.drop(['task_1','task_2','task_3','task_4'],axis=1)
df_ans.head()

,_id
0,Gujarati_image_1225.jpg
1,Gujarati_image_1583.jpg
2,Gujarati_image_1502.jpg
3,Gujarati_image_1487.jpg
4,Gujarati_image_1497.jpg


In [11]:
df_ans['label'] = pd.Series(pred_rft)
df_ans.to_csv('tfidf_rft3.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,1
1,Gujarati_image_1583.jpg,0
2,Gujarati_image_1502.jpg,0
3,Gujarati_image_1487.jpg,1
4,Gujarati_image_1497.jpg,1
5,Gujarati_image_1623.jpg,1
6,Gujarati_image_1493.jpg,1
7,Gujarati_image_1492.jpg,1
8,Gujarati_image_1462.jpg,0
9,Gujarati_image_1500.jpg,1


In [12]:
df_ans['label'] = pd.Series(pred_svm)
df_ans.to_csv('tfidf_svm3.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,1
1,Gujarati_image_1583.jpg,0
2,Gujarati_image_1502.jpg,0
3,Gujarati_image_1487.jpg,1
4,Gujarati_image_1497.jpg,0
5,Gujarati_image_1623.jpg,1
6,Gujarati_image_1493.jpg,0
7,Gujarati_image_1492.jpg,1
8,Gujarati_image_1462.jpg,0
9,Gujarati_image_1500.jpg,1


In [13]:
df_ans['label'] = pd.Series(pred_kn)
df_ans.to_csv('tfidf_knc3.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,1
1,Gujarati_image_1583.jpg,1
2,Gujarati_image_1502.jpg,0
3,Gujarati_image_1487.jpg,1
4,Gujarati_image_1497.jpg,1
5,Gujarati_image_1623.jpg,1
6,Gujarati_image_1493.jpg,1
7,Gujarati_image_1492.jpg,1
8,Gujarati_image_1462.jpg,0
9,Gujarati_image_1500.jpg,1


In [14]:
pred_df['res'] = pred_df.mode(axis=1)[0].astype(int)
pred_df.head()
res = pred_df['res'].tolist()

In [15]:
pred_df.head()

,KNeighbours,RandomForest,SVM,res
0,1,1,1,1
1,1,0,0,0
2,0,0,0,0
3,1,1,1,1
4,1,1,0,1


In [16]:
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Vulgar'
    else:
        res[i] = 'Non Vulgar'

In [17]:
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Vulgar
2,Gujarati_image_1502.jpg,Vulgar
3,Gujarati_image_1487.jpg,Non Vulgar
4,Gujarati_image_1497.jpg,Non Vulgar


In [18]:
df_ans.to_csv('tfidf_overall3.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Vulgar
2,Gujarati_image_1502.jpg,Vulgar
3,Gujarati_image_1487.jpg,Non Vulgar
4,Gujarati_image_1497.jpg,Non Vulgar
5,Gujarati_image_1623.jpg,Non Vulgar
6,Gujarati_image_1493.jpg,Non Vulgar
7,Gujarati_image_1492.jpg,Non Vulgar
8,Gujarati_image_1462.jpg,Vulgar
9,Gujarati_image_1500.jpg,Non Vulgar


In [19]:
res = pred_rft.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Vulgar'
    else:
        res[i] = 'Non Vulgar'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Vulgar
2,Gujarati_image_1502.jpg,Vulgar
3,Gujarati_image_1487.jpg,Non Vulgar
4,Gujarati_image_1497.jpg,Non Vulgar


In [20]:
df_ans.to_csv('tfidf_rft3.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Vulgar
2,Gujarati_image_1502.jpg,Vulgar
3,Gujarati_image_1487.jpg,Non Vulgar
4,Gujarati_image_1497.jpg,Non Vulgar
5,Gujarati_image_1623.jpg,Non Vulgar
6,Gujarati_image_1493.jpg,Non Vulgar
7,Gujarati_image_1492.jpg,Non Vulgar
8,Gujarati_image_1462.jpg,Vulgar
9,Gujarati_image_1500.jpg,Non Vulgar


In [21]:
res = pred_svm.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Vulgar'
    else:
        res[i] = 'Non Vulgar'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Vulgar
2,Gujarati_image_1502.jpg,Vulgar
3,Gujarati_image_1487.jpg,Non Vulgar
4,Gujarati_image_1497.jpg,Vulgar


In [22]:
df_ans.to_csv('tfidf_svm3.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Vulgar
2,Gujarati_image_1502.jpg,Vulgar
3,Gujarati_image_1487.jpg,Non Vulgar
4,Gujarati_image_1497.jpg,Vulgar
5,Gujarati_image_1623.jpg,Non Vulgar
6,Gujarati_image_1493.jpg,Vulgar
7,Gujarati_image_1492.jpg,Non Vulgar
8,Gujarati_image_1462.jpg,Vulgar
9,Gujarati_image_1500.jpg,Non Vulgar


In [23]:
res = pred_kn.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Vulgar'
    else:
        res[i] = 'Non Vulgar'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Non Vulgar
2,Gujarati_image_1502.jpg,Vulgar
3,Gujarati_image_1487.jpg,Non Vulgar
4,Gujarati_image_1497.jpg,Non Vulgar


In [24]:
df_ans.to_csv('tfidf_knc3.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Non Vulgar
1,Gujarati_image_1583.jpg,Non Vulgar
2,Gujarati_image_1502.jpg,Vulgar
3,Gujarati_image_1487.jpg,Non Vulgar
4,Gujarati_image_1497.jpg,Non Vulgar
5,Gujarati_image_1623.jpg,Non Vulgar
6,Gujarati_image_1493.jpg,Non Vulgar
7,Gujarati_image_1492.jpg,Non Vulgar
8,Gujarati_image_1462.jpg,Vulgar
9,Gujarati_image_1500.jpg,Non Vulgar


# task_4

In [5]:
import pickle
kn4= pickle.load(open('best_KN_model4.sav', 'rb'))
lr4= pickle.load(open('best_LR_model4.sav', 'rb'))
nb4= pickle.load(open('best_NB_model4.sav', 'rb'))
rf4= pickle.load(open('best_RF_model4.sav', 'rb'))
svm4= pickle.load(open('best_SVM_model4.sav', 'rb'))

In [6]:
X_test = X_test[:,:1741]

In [7]:
def transform(res):
    for i in range(len(res)):
        if (res[i] == 'Abusive'):
            res[i]= 0
        else:
            res[i]= 1
    return res

In [8]:
pred_df = pd.DataFrame() 
pred_kn = kn4.predict(X_test)
pred_kn = transform(pred_kn)
pred_df['KNeighbours'] = pred_kn
pred_rft = rf4.predict(X_test)
pred_rft = transform(pred_rft)
pred_df['RandomForest'] = pred_rft
pred_svm = svm4.predict(X_test)
pred_svm = transform(pred_svm)
pred_df['SVM'] = pred_svm

In [9]:
pred_kn.tolist().count(1),pred_rft.tolist().count(1),pred_svm.tolist().count(1)

(536, 556, 544)

In [10]:
df_ans = df_ans.drop(['text','text_clean'],axis=1)
df_ans = df_ans.reset_index()
df_ans = df_ans.drop(['index'],axis=1)
df_ans = df_ans.drop(['task_1','task_2','task_3','task_4'],axis=1)
df_ans.head()

,_id
0,Gujarati_image_1225.jpg
1,Gujarati_image_1583.jpg
2,Gujarati_image_1502.jpg
3,Gujarati_image_1487.jpg
4,Gujarati_image_1497.jpg


In [11]:
df_ans['label'] = pd.Series(pred_rft)
df_ans.to_csv('tfidf_rft4.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,1
1,Gujarati_image_1583.jpg,0
2,Gujarati_image_1502.jpg,0
3,Gujarati_image_1487.jpg,1
4,Gujarati_image_1497.jpg,1
5,Gujarati_image_1623.jpg,1
6,Gujarati_image_1493.jpg,1
7,Gujarati_image_1492.jpg,1
8,Gujarati_image_1462.jpg,1
9,Gujarati_image_1500.jpg,1


In [12]:
df_ans['label'] = pd.Series(pred_svm)
df_ans.to_csv('tfidf_svm4.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,1
1,Gujarati_image_1583.jpg,0
2,Gujarati_image_1502.jpg,0
3,Gujarati_image_1487.jpg,1
4,Gujarati_image_1497.jpg,1
5,Gujarati_image_1623.jpg,1
6,Gujarati_image_1493.jpg,0
7,Gujarati_image_1492.jpg,0
8,Gujarati_image_1462.jpg,1
9,Gujarati_image_1500.jpg,1


In [13]:
df_ans['label'] = pd.Series(pred_kn)
df_ans.to_csv('tfidf_knc4.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,1
1,Gujarati_image_1583.jpg,0
2,Gujarati_image_1502.jpg,1
3,Gujarati_image_1487.jpg,1
4,Gujarati_image_1497.jpg,1
5,Gujarati_image_1623.jpg,0
6,Gujarati_image_1493.jpg,1
7,Gujarati_image_1492.jpg,0
8,Gujarati_image_1462.jpg,1
9,Gujarati_image_1500.jpg,1


In [14]:
pred_df['res'] = pred_df.mode(axis=1)[0].astype(int)
pred_df.head()
res = pred_df['res'].tolist()

In [15]:
pred_df.head()

,KNeighbours,RandomForest,SVM,res
0,1,1,1,1
1,0,0,0,0
2,1,0,0,0
3,1,1,1,1
4,1,1,1,1


In [16]:
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Abusive'
    else:
        res[i] = 'Non-abusive'

In [17]:
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Abusive
3,Gujarati_image_1487.jpg,Non-abusive
4,Gujarati_image_1497.jpg,Non-abusive


In [18]:
df_ans.to_csv('tfidf_overall4.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Abusive
3,Gujarati_image_1487.jpg,Non-abusive
4,Gujarati_image_1497.jpg,Non-abusive
5,Gujarati_image_1623.jpg,Non-abusive
6,Gujarati_image_1493.jpg,Non-abusive
7,Gujarati_image_1492.jpg,Abusive
8,Gujarati_image_1462.jpg,Non-abusive
9,Gujarati_image_1500.jpg,Non-abusive


In [19]:
res = pred_rft.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Abusive'
    else:
        res[i] = 'Non-abusive'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Abusive
3,Gujarati_image_1487.jpg,Non-abusive
4,Gujarati_image_1497.jpg,Non-abusive


In [20]:
df_ans.to_csv('tfidf_rft4.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Abusive
3,Gujarati_image_1487.jpg,Non-abusive
4,Gujarati_image_1497.jpg,Non-abusive
5,Gujarati_image_1623.jpg,Non-abusive
6,Gujarati_image_1493.jpg,Non-abusive
7,Gujarati_image_1492.jpg,Non-abusive
8,Gujarati_image_1462.jpg,Non-abusive
9,Gujarati_image_1500.jpg,Non-abusive


In [21]:
res = pred_svm.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Abusive'
    else:
        res[i] = 'Non-abusive'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Abusive
3,Gujarati_image_1487.jpg,Non-abusive
4,Gujarati_image_1497.jpg,Non-abusive


In [22]:
df_ans.to_csv('tfidf_svm4.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Abusive
3,Gujarati_image_1487.jpg,Non-abusive
4,Gujarati_image_1497.jpg,Non-abusive
5,Gujarati_image_1623.jpg,Non-abusive
6,Gujarati_image_1493.jpg,Abusive
7,Gujarati_image_1492.jpg,Abusive
8,Gujarati_image_1462.jpg,Non-abusive
9,Gujarati_image_1500.jpg,Non-abusive


In [23]:
res = pred_kn.tolist()
for i in range(len(res)):
    if (res[i] == 0):
        res[i] = 'Abusive'
    else:
        res[i] = 'Non-abusive'
        
df_ans['label'] = pd.Series(res)
df_ans.head()

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Non-abusive
3,Gujarati_image_1487.jpg,Non-abusive
4,Gujarati_image_1497.jpg,Non-abusive


In [24]:
df_ans.to_csv('tfidf_knc4.csv',index=False)
df_ans.head(20)

,_id,label
0,Gujarati_image_1225.jpg,Non-abusive
1,Gujarati_image_1583.jpg,Abusive
2,Gujarati_image_1502.jpg,Non-abusive
3,Gujarati_image_1487.jpg,Non-abusive
4,Gujarati_image_1497.jpg,Non-abusive
5,Gujarati_image_1623.jpg,Abusive
6,Gujarati_image_1493.jpg,Non-abusive
7,Gujarati_image_1492.jpg,Abusive
8,Gujarati_image_1462.jpg,Non-abusive
9,Gujarati_image_1500.jpg,Non-abusive
